In [1]:
!pip install numpy matplotlib tensorflow

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

c:\Users\mural\OneDrive\Desktop\classification_proj\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [ ]:
pip install kaggle


In [ ]:

!kaggle datasets download -d anandkumarsahu09/cattle-breeds-dataset

In [7]:
!tar -xf cattle-breeds-dataset.zip


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values
    rotation_range=10,       # rotate images randomly
    width_shift_range=0.05,   # shift horizontally
    height_shift_range=0.05,  # shift vertically
    zoom_range=0.1,          # random zoomS
    horizontal_flip=True,    # flip horizontally
    validation_split=0.2     # 20% data for validation
)

# Training data
train_data = datagen.flow_from_directory(
    'cattle Breeds/',               # path to dataset folder
    target_size=(224,224),   # resize images
    batch_size=32,
    class_mode='categorical', # multi-class classification
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    'cattle Breeds/',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 969 images belonging to 5 classes.
Found 239 images belonging to 5 classes.


In [21]:
print("Training samples:", train_data.samples)
print("Validation samples:", val_data.samples)
num_classes = train_data.num_classes
print("Number of breeds:", num_classes)

Training samples: 969
Validation samples: 239
Number of breeds: 5


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = True


In [ ]:
for layer in base_model.layers[:100]:
    layer.trainable = False


In [33]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])


In [34]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [35]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(           #If validation loss does not improve for 5 consecutive epochs, stop training automatically
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(       #If validation loss stops improving, reduce the learning rate.”
        monitor="val_loss",
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 483ms/step - accuracy: 0.2394 - loss: 1.8861 - val_accuracy: 0.4184 - val_loss: 1.3967 - learning_rate: 1.0000e-04
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 382ms/step - accuracy: 0.4334 - loss: 1.3683 - val_accuracy: 0.5607 - val_loss: 1.1894 - learning_rate: 1.0000e-04
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 381ms/step - accuracy: 0.5635 - loss: 1.1658 - val_accuracy: 0.6109 - val_loss: 1.0952 - learning_rate: 1.0000e-04
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 378ms/step - accuracy: 0.6027 - loss: 1.0665 - val_accuracy: 0.6151 - val_loss: 1.0516 - learning_rate: 1.0000e-04
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 408ms/step - accuracy: 0.6202 - loss: 0.9810 - val_accuracy: 0.6276 - val_loss: 1.0027 - learning_rate: 1.0000e-04
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 385ms/step - accuracy: 0.6584 - loss: 0.9094 - val_accuracy: 0.6778 - val_loss: 0.9517 - learning_rate: 1.0000e-04
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 378ms/step - acc

In [36]:
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy after fine-tuning: {val_acc*100:.2f}%")


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 311ms/step - accuracy: 0.7155 - loss: 0.8858
Validation Accuracy after fine-tuning: 71.55%


In [37]:
y_true = val_data.classes
y_true


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=int32)

In [38]:
y_pred_probs = model.predict(val_data)
y_pred_probs


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 386ms/step


array([[0.75699574, 0.04436954, 0.01676648, 0.08680499, 0.09506319],
       [0.2693135 , 0.02058774, 0.65406924, 0.02962164, 0.0264079 ],
       [0.01477696, 0.04665373, 0.92356676, 0.00809037, 0.00691215],
       ...,
       [0.07650556, 0.02136838, 0.89016634, 0.00945158, 0.00250813],
       [0.1276019 , 0.07189045, 0.07235676, 0.5318192 , 0.19633166],
       [0.00637055, 0.9488899 , 0.00914002, 0.0300915 , 0.00550812]],
      shape=(239, 5), dtype=float32)

In [39]:
import numpy as np

y_pred = np.argmax(y_pred_probs, axis=1)
y_pred


array([0, 2, 2, 4, 4, 3, 2, 2, 2, 0, 3, 4, 4, 3, 0, 1, 3, 1, 1, 1, 2, 0,
       4, 2, 0, 0, 2, 1, 4, 2, 4, 0, 3, 4, 3, 1, 1, 4, 1, 4, 4, 0, 3, 0,
       4, 1, 1, 3, 0, 1, 2, 3, 3, 4, 2, 3, 1, 0, 0, 1, 2, 2, 4, 4, 2, 3,
       1, 3, 3, 1, 2, 0, 0, 1, 2, 4, 1, 1, 3, 4, 2, 0, 0, 0, 2, 0, 0, 2,
       1, 2, 1, 0, 4, 0, 1, 3, 0, 1, 3, 0, 2, 4, 4, 3, 4, 2, 3, 3, 3, 2,
       0, 3, 2, 0, 4, 0, 3, 3, 3, 1, 2, 1, 3, 2, 0, 2, 3, 2, 2, 2, 2, 4,
       4, 2, 1, 2, 4, 2, 0, 2, 4, 1, 3, 4, 4, 0, 4, 2, 0, 3, 0, 4, 4, 1,
       0, 1, 3, 2, 2, 4, 4, 2, 2, 0, 3, 0, 2, 3, 4, 4, 1, 0, 3, 0, 1, 4,
       4, 0, 1, 0, 3, 0, 4, 1, 2, 0, 0, 1, 0, 1, 0, 4, 4, 1, 0, 0, 2, 0,
       1, 4, 3, 2, 4, 3, 4, 2, 2, 1, 0, 0, 0, 0, 3, 4, 4, 1, 1, 3, 0, 0,
       4, 1, 0, 1, 1, 1, 2, 2, 1, 3, 1, 0, 2, 3, 4, 0, 2, 3, 1])

In [40]:
!pip install scikit-learn
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)


[[10 11 10  9 12]
 [12 11 10  8  6]
 [ 8  4 16 11 11]
 [16 10  7  6 11]
 [ 9 10  7  7  7]]


In [17]:
print(val_data.class_indices)


{'Ayrshire cattle': 0, 'Brown Swiss cattle': 1, 'Holstein Friesian cattle': 2, 'Jersey cattle': 3, 'Red Dane cattle': 4}


In [41]:
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    target_names=list(val_data.class_indices.keys())
))


                          precision    recall  f1-score   support

         Ayrshire cattle       0.18      0.19      0.19        52
      Brown Swiss cattle       0.24      0.23      0.24        47
Holstein Friesian cattle       0.32      0.32      0.32        50
           Jersey cattle       0.15      0.12      0.13        50
         Red Dane cattle       0.15      0.17      0.16        40

                accuracy                           0.21       239
               macro avg       0.21      0.21      0.21       239
            weighted avg       0.21      0.21      0.21       239

